# Date time object - Time Shifting/Lagging

Here are example implementations of Time Shifting/Lagging using Pandas, primarily using the .shift() method. This operation is crucial for analyzing trends, calculating differences between periods, and building features for forecasting.

* Time Shifting/Lagging with .shift()

The .shift() method shifts the data in a Series or DataFrame along the time axis by a specified number of periods. This means it moves the values either forward (creating lag) or backward (creating lead) relative to their original index.
  * periods: The number of periods (rows) to shift.
     * Positive periods (e.g., 1): Creates a lag (moves data forward in time, aligning with previous index value).
     * Negative periods (e.g., -1): Creates a lead (moves data backward in time, aligning with next index value).
  * freq: (Optional) If provided, this argument shifts the date index itself rather than just the rows. This is more robust for irregular time series as it respects calendar days/months/etc. rather than just row numbers. When freq is used, periods is ignored.

# 1. Basic Period Shift (Lagging and Leading) & Calculating Differences (Period-over-Period Change)

This demonstrates how to get previous day's data (lag) or next day's data (lead)

Shifting is commonly used as an intermediate step to calculate differences or percentage changes between consecutive time periods.

### A. Create the dataset

In [106]:
import pandas as pd
import numpy as np

# Create a DataFrame with daily stock prices

daily_prices = pd.DataFrame({
    'Close_Price': np.random.randint(100, 150, size=7)
}, index=pd.date_range(start='2025-06-01', periods=7, freq='D'))

daily_prices

,Close_Price
2025-06-01,103
2025-06-02,125
2025-06-03,128
2025-06-04,121
2025-06-05,130
2025-06-06,118
2025-06-07,139


### B. Basic Period Shift - Lag by 1 day (shift forward, so current day gets previous day's value)

In [107]:
# Lag by 1 day (shift forward, so current day gets previous day's value)

print("Lag by 1 Day")
daily_prices['Lag_1_Day'] = daily_prices['Close_Price'].shift(1)

daily_prices

Lag by 1 Day


,Close_Price,Lag_1_Day
2025-06-01,103,NaN
2025-06-02,125,103.0
2025-06-03,128,125.0
2025-06-04,121,128.0
2025-06-05,130,121.0
2025-06-06,118,130.0
2025-06-07,139,118.0


Note: The first row's 'Lag_1_Day' is NaN because there's no data before '2025-06-01'.

### C. Basic Period Shift - Lead by 1 day (shift backward, so current day gets next day's value)

In [108]:
# Lead by 1 day (shift backward, so current day gets next day's value)

print(" Lead by 1 Day ")
daily_prices['Lead_1_Day'] = daily_prices['Close_Price'].shift(-1)

daily_prices

 Lead by 1 Day 


,Close_Price,Lag_1_Day,Lead_1_Day
2025-06-01,103,NaN,125.0
2025-06-02,125,103.0,128.0
2025-06-03,128,125.0,121.0
2025-06-04,121,128.0,130.0
2025-06-05,130,121.0,118.0
2025-06-06,118,130.0,139.0
2025-06-07,139,118.0,NaN


Note: The last row's 'Lead_1_Day' is NaN because there's no data after '2025-06-07'.

### D. Calculating Differences"

In [109]:
print("Calculating Differences")

# Using the same daily_prices DataFrame from above
# Calculate the absolute daily change
daily_prices['Daily_Change'] = daily_prices['Close_Price'] - daily_prices['Close_Price'].shift(1)

daily_prices

Calculating Differences


,Close_Price,Lag_1_Day,Lead_1_Day,Daily_Change
2025-06-01,103,NaN,125.0,NaN
2025-06-02,125,103.0,128.0,22.0
2025-06-03,128,125.0,121.0,3.0
2025-06-04,121,128.0,130.0,-7.0
2025-06-05,130,121.0,118.0,9.0
2025-06-06,118,130.0,139.0,-12.0
2025-06-07,139,118.0,NaN,21.0


### E. Calculate the percentage daily change

In [110]:
# Calculate the percentage daily change

daily_prices['Daily_Pct_Change'] = daily_prices['Close_Price'].pct_change(periods=1)
# pd.Series.pct_change() is a specialized function built on shift()

daily_prices

,Close_Price,Lag_1_Day,Lead_1_Day,Daily_Change,Daily_Pct_Change
2025-06-01,103,NaN,125.0,NaN,NaN
2025-06-02,125,103.0,128.0,22.0,0.213592
2025-06-03,128,125.0,121.0,3.0,0.024000
2025-06-04,121,128.0,130.0,-7.0,-0.054688
2025-06-05,130,121.0,118.0,9.0,0.074380
2025-06-06,118,130.0,139.0,-12.0,-0.092308
2025-06-07,139,118.0,NaN,21.0,0.177966


# 2. Time-based Shift using freq (More Robust for Irregular Data)

In [115]:
import pandas as pd
import numpy as np

print("Time-based Shift using 'freq'")

# Create an irregular daily DataFrame (e.g., missing weekend data)
irregular_dates = pd.to_datetime(['2025-06-02', '2025-06-03', '2025-06-04', '2025-06-05', '2025-06-06', '2025-06-09', '2025-06-10'])
irregular_data = pd.DataFrame({
    'Value': np.random.randint(20, 50, size=len(irregular_dates))
}, index=irregular_dates)

irregular_data

Time-based Shift using 'freq'


,Value
2025-06-02,41
2025-06-03,32
2025-06-04,36
2025-06-05,29
2025-06-06,22
2025-06-09,24
2025-06-10,36


In [116]:
# Shift by 1 day using 'freq'
print("Shift by 1 Day (freq='D')")
# This shifts the index by 1 day, aligning the value of 2025-06-02 with 2025-06-03
# It correctly handles the gap for the weekend

irregular_data['Value_Shifted_1D_Freq'] = irregular_data['Value'].shift(freq='D')
print(irregular_data)
# Notice how the value for 2025-06-02 (e.g., 20) now appears on 2025-06-03.
# The row for 2025-06-09 gets the value from 2025-06-06, skipping the weekend correctly.

# Shift by 1 period (row-based) for comparison
print("Shift by 1 Period (no freq) for comparison")
# This shifts by row count, not by actual time.
# The value for 2025-06-02 (e.g., 20) appears on 2025-06-03.
# The value for 2025-06-06 appears on 2025-06-09.

irregular_data['Value_Shifted_1Period'] = irregular_data['Value'].shift(1)
print(irregular_data)
# Observe the difference for the weekend gap. shift(freq='D') maintains the actual time shift,
# while shift(1) just moves the value to the next row, regardless of the time gap.

Shift by 1 Day (freq='D')
            Value  Value_Shifted_1D_Freq
2025-06-02     41                    NaN
2025-06-03     32                   41.0
2025-06-04     36                   32.0
2025-06-05     29                   36.0
2025-06-06     22                   29.0
2025-06-09     24                    NaN
2025-06-10     36                   24.0
Shift by 1 Period (no freq) for comparison
            Value  Value_Shifted_1D_Freq  Value_Shifted_1Period
2025-06-02     41                    NaN                    NaN
2025-06-03     32                   41.0                   41.0
2025-06-04     36                   32.0                   32.0
2025-06-05     29                   36.0                   36.0
2025-06-06     22                   29.0                   29.0
2025-06-09     24                    NaN                   22.0
2025-06-10     36                   24.0                   24.0


# COMPLETED